In [ ]:
import numpy as np
import os
import torch
from MSDataset import MSDataset
from Model import MSNetEncoder, MSNetAutoEncoder
from multiprocessing import Pool
from pyteomics import mzml

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = MSDataset("data", Pool(16), device)

In [ ]:
autoencoder = MSNetAutoEncoder().to(device)

In [ ]:
def train(model, loss_fn, dataset):
    n = len(dataset.samples)
    optim = torch.optim.SGD(model.parameters(), lr=0.00002, momentum=0.4)
    for epoch in range(20):
        sample = torch.randint(n, size=(n,)).tolist()
        total_loss = 0
        for idx in sample:
            optim.zero_grad()
            X = dataset[idx][0]
            X_pred = model(X)
            loss = loss_fn(X, X_pred)
            loss.backward()
            optim.step()
            total_loss = total_loss + loss.item()
        print(f"Epoch {epoch + 1} has training loss {total_loss / n}")
        
        

In [ ]:
train(autoencoder, torch.nn.HuberLoss(), dataset)